**ЧАСТЬ 1**

# Исследование надежности заемщиков

In [ ]:
import pandas as pd# импортируйте библиотеку pandas

In [ ]:
data = pd.read_csv('/datasets/data.csv')# прочитайте csv-файл

In [ ]:
print(data.head(20))# ваш код здесь

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

In [ ]:
data.info()# ваш код здесь

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [ ]:
data.isna().sum()# ваш код здесь

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [ ]:
#total_income_median = data.groupby('income_type')['total_income'].median()
for tot_inc in data['income_type'].unique():
    data.loc[(data['income_type']==tot_inc) & (data['total_income'].isna()), 'total_income'] = data.loc[(data['income_type'] == tot_inc), 'total_income'].median()

### Обработка аномальных значений

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода `abs()`.**

In [ ]:
data['days_employed'] = data['days_employed'].abs()# ваш код здесь

In [ ]:
print(data.groupby('income_type')['days_employed'].median())# ваш код здесь

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64


У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть.

In [ ]:
print(data['children'].unique()) # ваш код здесь

[ 1  0  3  2 -1  4 20  5]


**В столбце `children` есть два аномальных значения. Удаляем строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [ ]:
data = data[data['children'] != -1]
data = data[data['children'] != 20] # ваш код здесь

In [ ]:
data['children'].unique()# ваш код здесь

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**Заполняем пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.**

In [ ]:
for days_emp in data['income_type'].unique():
    data.loc[(data['income_type']==days_emp) & (data['days_employed'].isna()), 'days_employed'] = data.loc[(data['income_type'] == days_emp), 'days_employed'].median()# ваш код здесь

In [ ]:
print(data.isna().sum()) # ваш код здесь

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


### Изменение типов данных

**Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [ ]:
data['total_income'] = data['total_income'].astype(int) # ваш код здесь

### Обработка дубликатов

**Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.**

In [ ]:
data['education'] = data['education'].str.lower()# ваш код здесь

**Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [ ]:
print(data.duplicated().sum())# посчитайте дубликаты

71


In [ ]:
data = data.drop_duplicates()# удалите дубликаты

### Категоризация данных

**На основании диапазонов, указанных ниже, создаем в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [ ]:
def categorize_income(total_income): # создайте функцию categorize_income()
    if total_income<=30000:
        return 'E'
    elif 30001<=total_income<=50000:
        return 'D'
    elif 50001<=total_income<=200000:
        return 'C'
    elif 200001<=total_income<=1000000:
        return 'B'
    elif total_income>=1000001:
        return 'A'

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)# примените функцию методом apply()

**Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [ ]:
print(data['purpose'].unique())# ваш код здесь

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


**Создаем функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**

In [ ]:
def categorize_purpose(purpose): # создайте функцию categorize_purpose()
    if purpose in ['покупка жилья', 'операции с жильем', 'покупка жилья для семьи', 'покупка недвижимости', 'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
 'строительство собственной недвижимости','операции с коммерческой недвижимостью', 'недвижимость','строительство жилой недвижимости', 'строительство недвижимости', 'жилье', 'операции со своей недвижимостью', 'покупка своего жилья','операции с недвижимостью', 'покупка жилья для сдачи', 'ремонт жилью']:
        return 'операции с недвижимостью'
    if purpose in ['приобретение автомобиля', 'на покупку подержанного автомобиля', 'на покупку своего автомобиля', 'автомобили', 'сделка с подержанным автомобилем', 'автомобиль', 'свой автомобиль', 'сделка с автомобилем', 'на покупку автомобиля']:
        return 'операции с автомобилем'
    if purpose in ['сыграть свадьбу',  'на проведение свадьбы', 'свадьба']:
        return 'проведение свадьбы'
    else:
        return 'получение образования'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)# примените функцию методом apply()

**ЧАСТЬ 2**

In [ ]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [ ]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
data.isna().sum() # Посмотрим число пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

 Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится нам для исследования.

In [ ]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [ ]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [ ]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [ ]:
data['total_income'] = data['total_income'].astype(int)

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру. Проверим:

In [ ]:
data['education'] = data['education'].str.lower()

In [ ]:
data.duplicated().sum()

71

In [ ]:
data = data.drop_duplicates()

**Категоризация данных**

На основании диапазонов, указанных ниже, создаем в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`.

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [ ]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создаем функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.

Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут нам правильно определить категорию.

In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

Есть ли зависимость между количеством детей и возвратом кредита в срок?

Проверим значения в столбце debt на наличие случайных или ошибочных значений и создадим сводную таблицу со случаями просрочки и общим количеством выданных кредитов:

In [ ]:
print(data['debt'].unique()) # убедимся, что в столбце о наличии задолженности нет "случайных" значений, т.е. только 0 или 1.

# сгруппируем по наличию детей,применим аггрегирующие функции к столбцу:
children_debt_quantity = data.groupby('children').agg({'debt':['sum','count']})
print(children_debt_quantity) #выводим результат

[0 1]
          debt       
           sum  count
children             
0         1063  14091
1          444   4808
2          194   2052
3           27    330
4            4     41
5            0      9


Убираем индексы и мультииндексы:

In [ ]:
children_debt_quantity.columns = ['debt_quantity', 'user_total'] # избавляемся от мультииндекса в столбцах
children_debt_quantity = children_debt_quantity.reset_index(inplace=False) #убираем столбец children из мультииндекса
print(children_debt_quantity) # проверка

   children  debt_quantity  user_total
0         0           1063       14091
1         1            444        4808
2         2            194        2052
3         3             27         330
4         4              4          41
5         5              0           9


Добавляем колонку debt_percent - отношение случаев просрочки к общему числу выданных кредитов:

In [ ]:
# создадим столбец отношения кол-во просрочек(в зависимости от наличия детей) к общему количеству кредитов
children_debt_quantity['debt_percent'] = children_debt_quantity['debt_quantity']/children_debt_quantity['user_total']

# для наглядности переведем в проценты и округлим значения до 2 знаков после запятой
children_debt_quantity['debt_percent'] = (children_debt_quantity['debt_percent']*100).round(2)
print(children_debt_quantity) # выводим результат

   children  debt_quantity  user_total  debt_percent
0         0           1063       14091          7.54
1         1            444        4808          9.23
2         2            194        2052          9.45
3         3             27         330          8.18
4         4              4          41          9.76
5         5              0           9          0.00


Далее может быть два пути (оба ведут к +/- одинковым результатам (проверил)):
    1) Либо мы дропаем строку с людьми имеющими 5 детей - т.к. их количество ничтожно мало по сравнению с остальной выборкой (т.е. 0 в графе просрочек - это "выброс") и делаем сводную таблицу с категориями "есть/нет дети" и считаем процент просрочки в категориях по среднему;
    2) Либо мы оставляем строку с 5 детьми, делаем сводную таблицу с категориями "есть/нет дети", а значения процента просрочек в категориях считаем по медиане (в соответствии с рекомендациями курса ЯндексПрактикум по количественным значениям)
    Пойдем вторым путем:

Разобьем на категории с наличием и отсутствием детей:

In [ ]:
def has_child(children): #напишем функцию has_child(children):
    if 5>=children>=1:
        return 'есть дети'
    return 'нет детей'

#применим функцию к сводной таблице children_debt_quantity:
children_debt_quantity['has_child'] = children_debt_quantity['children'].apply(has_child)
print(children_debt_quantity) # Проверим что получилось

   children  debt_quantity  user_total  debt_percent  has_child
0         0           1063       14091          7.54  нет детей
1         1            444        4808          9.23  есть дети
2         2            194        2052          9.45  есть дети
3         3             27         330          8.18  есть дети
4         4              4          41          9.76  есть дети
5         5              0           9          0.00  есть дети


Создадим  сводную таблицу в зависимости от has_child:

In [ ]:
# создаем таблицу и считаем медиану в каждой категории
has_child_df = children_debt_quantity.groupby('has_child')['debt_percent'].median()
print(has_child_df)

has_child
есть дети    9.23
нет детей    7.54
Name: debt_percent, dtype: float64



Судя по таблице has_child_df сам факт наличия детей в некоторой степени повышает риск задолженности по кредиту ( в среднем на ~2%). Данные по людям с 5 детьми в данном случае представляют собой выброс, поэтому значение в категориях о наличии детей считалось по медиане (не только поэтому, но и из-за того что количественные переменные лучше считать по медиане).
Как вариант - убрать из исследования данные по людям с 3-5 детьми, так как они составляют менее 2% выборки.
В данном случае, исследование проводилось с использованием этих данных.

Есть ли зависимость между семейным положением и возвратом кредита в срок?

Значения в столбце debt мы проверили ранее, поэтому сразу создадим сводную таблицу со случаями просрочки и общим количеством выданных кредитов:

In [ ]:
# сгруппируем по семейному положению, применим аггрегирующие функции к столбцу
family_status_debt = data.groupby('family_status').agg({'debt':['sum','count']})
print(family_status_debt) #выводим результат

                      debt       
                       sum  count
family_status                    
Не женат / не замужем  273   2796
в разводе               84   1189
вдовец / вдова          63    951
гражданский брак       385   4134
женат / замужем        927  12261


Убираем индексы и мультииндексы:

In [ ]:
family_status_debt.columns = ['debt_quantity', 'user_total'] # избавляемся от мультииндекса в столбцах
family_status_debt = family_status_debt.reset_index(inplace=False) #убираем столбец family_status из индекса
print(family_status_debt) # проверка

           family_status  debt_quantity  user_total
0  Не женат / не замужем            273        2796
1              в разводе             84        1189
2         вдовец / вдова             63         951
3       гражданский брак            385        4134
4        женат / замужем            927       12261


Cоздаем стобец отношения, переводим в проценты, округляем:

In [ ]:
#создаем стобец отношения, переводим в проценты, округляем
family_status_debt['debt_percent'] = ((family_status_debt['debt_quantity']/family_status_debt['user_total'])*100).round(2)
print(family_status_debt)


           family_status  debt_quantity  user_total  debt_percent
0  Не женат / не замужем            273        2796          9.76
1              в разводе             84        1189          7.06
2         вдовец / вдова             63         951          6.62
3       гражданский брак            385        4134          9.31
4        женат / замужем            927       12261          7.56


Полученные результаты можно доработать, разбив группы по категориям "есть/нет партнер":

In [ ]:
#для этого напишем функцию partner_categorize:
def partner_categorize(family_status):
    if family_status in ['Не женат / не замужем', 'в разводе', 'вдовец / вдова']:
        return 'нет партнера'
    elif family_status in ['гражданский брак', 'женат / замужем']:
        return 'есть партнер'

#создадим столбец is_partner в датафрейме family_status_debt и применим написанную функцию:
family_status_debt['is_partner'] = family_status_debt['family_status'].apply(partner_categorize)
print(family_status_debt) #выведем данные на экран

           family_status  debt_quantity  user_total  debt_percent  \
0  Не женат / не замужем            273        2796          9.76   
1              в разводе             84        1189          7.06   
2         вдовец / вдова             63         951          6.62   
3       гражданский брак            385        4134          9.31   
4        женат / замужем            927       12261          7.56   

     is_partner  
0  нет партнера  
1  нет партнера  
2  нет партнера  
3  есть партнер  
4  есть партнер  


Сгруппируем данные в зависимости от наличия/отсутствия партнера:

In [ ]:
#сгруппируем данные, применим посчитаем среднее в каждой группе (в данном случае )
is_partner_df = family_status_debt.groupby('is_partner')['debt_percent'].median().round(2)
print(is_partner_df) #выведем результат на экран

is_partner
есть партнер    8.44
нет партнера    7.06
Name: debt_percent, dtype: float64


Можно также посмотреть зависимость просрочки от параметра "состоял ли когда-нибудь в браке":

In [ ]:
#для этого напишем функцию was_married:
def was_married(family_status):
    if family_status in ['Не женат / не замужем', 'гражданский брак']:
        return 'не состоял(а) в браке'
    elif family_status in ['женат / замужем','в разводе', 'вдовец / вдова']:
        return 'состоит или состоял(а) в браке'
#создадим столбец was_married в датафрейме family_status_debt и применим написанную функцию:
family_status_debt['was_married'] = family_status_debt['family_status'].apply(was_married)
print(family_status_debt) #выведем данные на экран

           family_status  debt_quantity  user_total  debt_percent  \
0  Не женат / не замужем            273        2796          9.76   
1              в разводе             84        1189          7.06   
2         вдовец / вдова             63         951          6.62   
3       гражданский брак            385        4134          9.31   
4        женат / замужем            927       12261          7.56   

     is_partner                     was_married  
0  нет партнера           не состоял(а) в браке  
1  нет партнера  состоит или состоял(а) в браке  
2  нет партнера  состоит или состоял(а) в браке  
3  есть партнер           не состоял(а) в браке  
4  есть партнер  состоит или состоял(а) в браке  


In [ ]:
#сгруппируем данные, применим посчитаем среднее в каждой группе (в данном случае )
was_married_df = family_status_debt.groupby('was_married')['debt_percent'].mean().round(2)
print(was_married_df) #выведем результат на экран

was_married
не состоял(а) в браке             9.54
состоит или состоял(а) в браке    7.08
Name: debt_percent, dtype: float64


Если брать в широком смысле, то наличие партнера, как ни странно немного повышает риск возникновения задолженности по кредиту (на ~1,5%), тем не менее сам факт того, что человек состоит или состоял в браке снижает этот показатель (на 2,5%).
Если брать для анализа более узкие данные, то вдовы/вдовцы наиболее ответственные заёмщики, за ними следом с отставанием 0,5-1 % идут замужем/женатые или в разводе. Люди не женатые/не замужем или состоящие в гражданском браке являются наименее ответственными заемщиками, процент случаев просрочки выплат приближается к 10%, что на 3 % выше чем у вдов/вдовцов и на 2% выше чем у людей в разводе или замужем/женатых.

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Создадим сводную таблицу:

In [ ]:
#Сгруппируем по категории зарплат и выведем данные:
income_category_df = data.groupby('total_income_category').agg({'debt':['sum','count']})
print(income_category_df)

                       debt       
                        sum  count
total_income_category             
A                         2     25
B                       354   5014
C                      1353  15921
D                        21    349
E                         2     22


Для наглядности таблицы, переименуем категории в диапазоны доходов, для этого сбросим индекс и напишем функцию:

In [ ]:
income_category_df = income_category_df.reset_index() #сбросим индекс
income_category_df.columns = ['total_income_category', 'debt_quantity', 'user_total'] # избавляемся от мультииндекса в столбцах
print(income_category_df) # проверка

  total_income_category  debt_quantity  user_total
0                     A              2          25
1                     B            354        5014
2                     C           1353       15921
3                     D             21         349
4                     E              2          22


In [ ]:
#запишем функцию income_range(total_income_category):
def income_range(total_income_category):
    if total_income_category == 'A':
        return 'свыше 1000000'
    if total_income_category == 'B':
        return '200001 ... 1000000'
    if total_income_category == 'C':
        return '50001 ... 200000'
    if total_income_category == 'D':
        return '30001 ... 50000'
    if total_income_category == 'E':
        return 'менее 30000'

применим функцию к столбцу total_income_category датафрейма income_category_df:

In [ ]:
#применим функцию к столбцу total_income_category
income_category_df['total_income_category'] = income_category_df['total_income_category'].apply(income_range)

#добавим столбец с процентами
income_category_df['percent'] = ((income_category_df['debt_quantity']/income_category_df['user_total'])*100).round(2)
print(income_category_df)

  total_income_category  debt_quantity  user_total  percent
0         свыше 1000000              2          25     8.00
1    200001 ... 1000000            354        5014     7.06
2      50001 ... 200000           1353       15921     8.50
3       30001 ... 50000             21         349     6.02
4           менее 30000              2          22     9.09


Если брать в расчет только данные по которым имеется достаточное количество значений (категории "200000-1000000" и "50001-200000", что составляет 98% от выборки), то можно сделать вывод о том, что рост абсолютного дохода напрямую влияет на вероятность просрочки по кредиту.
Группы с доходами ниже 50000 и свыше 1000000 в данном случае имеют маленькую выборку - данными от них можно пренебречь.
Если же необходимо из подгрупп выделить наиболее надёжного заёмщика - то это будут люди с доходом в 30000-50000, они не платят в срок лишь в 6% случаев.**

Альтернативное решение через qcut():

Для начала отсортируем данные по доходу в исходном датафрейме и запишем данные в новый датафрейм. Это нужно для того,
чтобы когда мы будем подклеивать значения debt к разбитому на квантили датафрейму - не нарушить порядок.

In [ ]:
data_income_sort = data.sort_values(by='total_income') #сортируем
#print(data_income_sort.head(10)) #проверяем

Применяем метод qcut() для разбития выборки на равные части:

In [ ]:
income_categories = pd.qcut(data_income_sort['total_income'], q=5) #разбиваем данные на 5 равных по количеству частей

#смотрим количество значений в каждой части и создаем датафрейм:
income_cat_val = income_categories.value_counts()
income_cat_val = income_cat_val.reset_index() #сбрасываем индекс
#для наглядности переименуем столбцы
income_cat_val = income_cat_val.rename(columns={'index': 'income_range', 'total_income': 'q_part'})
print(income_cat_val) #проверяем

            income_range  q_part
0   (20666.999, 98514.0]    4267
1    (98514.0, 132113.0]    4266
2   (132113.0, 161380.0]    4266
3   (161380.0, 214604.0]    4266
4  (214604.0, 2265604.0]    4266


In [ ]:
data_income_sort

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
14585,0,359219.059341,57,среднее,1,женат / замужем,0,F,пенсионер,1,20667,недвижимость,E,операции с недвижимостью
13006,0,369708.589113,37,среднее,1,гражданский брак,1,M,пенсионер,0,21205,заняться высшим образованием,E,получение образования
16174,1,3642.820023,52,среднее,1,женат / замужем,0,M,сотрудник,0,21367,приобретение автомобиля,E,операции с автомобилем
1598,0,359726.104207,68,среднее,1,гражданский брак,1,M,пенсионер,0,21695,на проведение свадьбы,E,проведение свадьбы
14276,0,346602.453782,61,среднее,1,женат / замужем,0,F,пенсионер,0,21895,недвижимость,E,операции с недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17178,0,5734.127087,42,высшее,0,гражданский брак,1,M,компаньон,0,1711309,сыграть свадьбу,A,проведение свадьбы
20809,0,4719.273476,61,среднее,1,Не женат / не замужем,4,F,сотрудник,0,1715018,покупка жилья для семьи,A,операции с недвижимостью
9169,1,5248.554336,35,среднее,1,гражданский брак,1,M,сотрудник,0,1726276,дополнительное образование,A,получение образования
19606,1,2577.664662,39,высшее,0,женат / замужем,0,M,компаньон,1,2200852,строительство недвижимости,A,операции с недвижимостью


In [ ]:
#приклеиваем к income_categories значения столбца debt из датафрейма data_income_sort
income_categories = pd.concat([income_categories, data_income_sort['debt']], axis=1)

Группируем по столбцу total_income, складываем количество просрочек и заводим новый столбец с процентом просрочек:

In [ ]:
#группируем по столбцу total_income, складываем количество просрочек
income_cat_df = income_categories.groupby('total_income')['debt'].sum() # вот здесь, если применить agg({'debt':['sum','count']})
#то выходит ошибка, поэтому дальнейшее решение по расчетам думаю верное, но по коду - не уверен.

In [ ]:
income_cat_df = income_cat_df.reset_index() #сбрасываем индекс
#для наглядности переименуем столбцы
income_cat_df = income_cat_df.rename(columns={'total_income': 'income_range', 'debt': 'q_debt'})

In [ ]:
#заводим новый столбец со значением отношением просрочка/общее количество:
income_cat_df['debt_percent'] = ((income_cat_df['q_debt']/income_cat_val['q_part'])*100).round(2)
print(income_cat_df) # проверяем

            income_range  q_debt  debt_percent
0   (20666.999, 98514.0]     344          8.06
1    (98514.0, 132113.0]     358          8.39
2   (132113.0, 161380.0]     373          8.74
3   (161380.0, 214604.0]     358          8.39
4  (214604.0, 2265604.0]     299          7.01


Вывод по исследованию через qcut():
Выявлено, что люди с высоким уровнем дохода (свыше 214000) реже других совершают просрочки по кредитам (на 1-1,5%). При этом не выявлено достоверной связи между ростом абсолютного дохода и снижением просрочек по платежам. Люди из группы с доходом 20000-99000 платят кредиты исправнее, чем люди с доходом 130000-160000.

Как разные цели кредита влияют на его возврат в срок?

In [ ]:
purpose_category_df = data.groupby('purpose_category').agg({'debt':['sum', 'count']}) # Действия аналогичные предыдущему заданию
#print(purpose_category_df) # проверка

purpose_category_df = purpose_category_df.reset_index() # убираем индекс
purpose_category_df.columns = ['purpose_category', 'debt_quantity', 'user_total'] # избавляемся от мультииндекса и переименовываем столбцы
#print(purpose_category_df) #проверка

#создаем столбец с процентами:
purpose_category_df['percent'] = ((purpose_category_df['debt_quantity']/purpose_category_df['user_total'])*100).round(2)
print(purpose_category_df)

           purpose_category  debt_quantity  user_total  percent
0    операции с автомобилем            400        4279     9.35
1  операции с недвижимостью            780       10751     7.26
2     получение образования            369        3988     9.25
3        проведение свадьбы            183        2313     7.91


Есть взаимосвязь между целью кредита и вероятностью возникновения задолженности. Так, люди берущие кредит на операции с автомобилем и получение образования сталкиваются с невозможностью вовремя произвести платёж чаще, чем люди, цель кредита которых являются операции с недвижимостью и проведение свадьбы. Разница между двумя этими "подкатегориями" составляет ~1,5-2%.**

Думаю, что недвижимостью занимаются люди с большей дисциплинированностью и возможно большим доходом. Стимул выплатить ипотеку выше - т.к. остаться без крыши над головой - это не езда на общественном транспорте (как в случае с просрочкой платежей по кредиту на автомобиль). Процент по ипотеке ниже чем во всех остальных видах кредита - это тоже сказывается.
В случае же свадьб, полагаю, что деньги полученные в качестве подарка на свадьбу идут в счет покашения кредита, что сильно облегчает долговую нагрузку.

Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* Пропуски в графе days_employed у пенсионеров, студентов, безработных и в декрете - неслучайны. Данные категории могут не иметь работы. Пенсионеры и студенты бывают работающие, но в данном случае, вероятно число дней не указано из-за отсутвия работы. У госслужащих, возможно, пропуски в данной графе также могут быть неслучайны, т.к. инфомация о службе может быть обьектом гостайны. У остальных, скорее всего пропуски носят случайных характер и связаны с техническими ошибками.
Отсутствие данных по доходу также может быть связано со столбцом income_type. Безработные,  студенты и, возможно, родители в декрете могут не иметь официального источника дохода (имеется в виду, что их "кормят" члены их семей, а не то что они совсем без денег сидят). А вот у остальных данные по доходу скорее всего отсутствуют из-за небрежности заполнения или технической ошибки, т.к. вероятность того, что человеку выдадут кредит без получения от него информации о доходе крайне низкая.

Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* В случае зарплат или других подобных переменных среди значений с плавно меняющимися количественными параметрами могут попадаться "выбросы" - единичные, сильно отличающиеся от остальных значения. Это не ошибки, а реальные данные (хотя ошибки тоже нельзя исключить), которые могут сильно "подвинуть" среднее значение, что при заполнении пропусков может исказить общую картину данных по выбранному параметру (в нашем случае зарплате).

ВЫВОД
Прежде чем перейти к исследованию, пришлось провести небольшую предобработку данных, она включала:
- работа с аномальными значениями (стаж и количество детей местами были отрицательны);
- обработка пропусков (замена медианными значениями в стаже в зависимости от категории занятости);
- удалены дубликаты.

Исходя из проведенного исследования, можно сделать следующие выводы:
- Наличие детей приводит к увеличению вероятности просрочки по кредиту (на 1-2%);

- Семейное положение также влияет на просрочку по кредиту: самые надежные заёмщики - вдовцы и вдовы (всего 6% случаев просрочки), самые ненадёжные - не состоящие в браке или живущие в гражданском браке (процент просрочки по кредиту приближается к 10%); При этом наличие партнера исходя из сводных данных - повышает вероятность просрочки(разница между категориями составляет ~1.5%); Если же посмотреть на распределение по категориям "состоял(а) ли когда-либо в браке" или "не состоял(а)" то можно заметить что люди из первой категории реже допускают просрочки платежей (разница достигает 2,5%);

- Исходя из двух видов проведенных исследований, можно сказать, что размер дохода влиет на процент просрочек. Зависимость нельзя назвать прямой, тем не менее, она есть. При этом люди с небольшим доходом (около 50000) могут справляться с обязательствами по кредиту лучше тех, кто имеет средние значения (около 150000). При этом согласно альтернативному исследованию (через qcut()) люди имеющие доход свыше 214000 значительно пунктуальнее в выплатах относительно других подгрупп (разница до 1,5%)
.
- Люди берущие кредит на операции с авто и получение образования чаще нарушают график платежей, по сравнению с теми, кто берет деньги на свадьбу или жилье (9% и 7% соответственно).

Таким образом, одинокие вдовы/вдовцы, без детей, с доходом свыше 214000, берущие кредит на свадьбу или на жилье чаще всех остальных соблюдают график платежей.

А вот живущие в гражданском браке, с детьми, имеющие средний или доход ниже среднего берущие деньги на образование или авто - напротив - наиболее вероятный должник по кредиту.